In [28]:
#!pip install sktime --user

In [29]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

sns.set(style="whitegrid", palette="Dark2")

from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv('../../Data/Processed/Occupancy_processed.csv', parse_dates=['date'])
df.set_index('date', inplace=True)

In [31]:
X = df.drop(['Occupancy'], axis=1)
y = df.Occupancy

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [33]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(15420, 5) (15420,) (5140, 5) (5140,)
